## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Creación de nuevas características (feature engineering).
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.



In [1]:
## Librerías
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
import os

In [2]:
#!pip install category_encoders

In [3]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##  Cargamos los Dataset

- Trabajaremos exclusivamente con el dataset de entrenamiento "train_transaction" para obtener tanto los datos de entrenamiento como los de validación. De este dataset solo tomaremos el 10 % del total de los registro.

In [4]:
select_col_transaction = 'TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9'.split(',')
select_col_identity = 'TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo'.split(',')
select_col_transaction_test = 'TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9'.split(',')
#select_col_identity_test ='TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo'.split(',')

In [5]:
col_range_10 =  ['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321']

### Selección de porción del Dataset de entrenamiento con Muestreo Estratificado



In [6]:
#data_dir = '../data/raw/ieee-fraud-detection'

## dataset de entrenamiento
seed = 42
# Cargar los datos
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv', usecols=col_range_10)
#df_identity_train = pd.read_csv(data_dir + '/train_identity.csv', usecols=select_col_identity)

# Combinar los datasets
#dataset = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')
dataset = df_transaction_train.copy()
# Realizar el muestreo estratificado
train_data, _ = train_test_split(dataset, stratify=dataset['isFraud'], test_size=0.9, random_state=seed)

# Mostrar la información del dataset resultante
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59054 entries, 294161 to 404259
Columns: 112 entries, TransactionID to V321
dtypes: float64(105), int64(4), object(3)
memory usage: 50.9+ MB


### Armamos un nuevo dataset con las columnas más relevantes

In [7]:
# data = data[['isFraud', 'TransactionDT', 'TransactionAmt',
#        'ProductCD','addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
#        'DeviceType', 'DeviceInfo','card1', 'card2', 'card3', 'card4', 'card5', 'card6','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
#        'C12', 'C13', 'C14',]].copy()
data = train_data.copy()

## Manejo de valores nulos

- Basándonos en el análisis exploratorio de datos (EDA) realizado previamente, eliminamos las columnas que contienen más del 80% de valores nulos.
- Eliminamos las filas con valores nulos debido a que tenemos suficientes cantidad de registros

In [8]:
#data.drop(['DeviceType','R_emaildomain','DeviceInfo','dist1','dist2'], axis=1,inplace=True)

In [9]:
data.dropna(inplace=True)

In [10]:
data.isnull().sum()

TransactionID     0
isFraud           0
TransactionDT     0
TransactionAmt    0
ProductCD         0
                 ..
V317              0
V318              0
V319              0
V320              0
V321              0
Length: 112, dtype: int64

In [11]:
null_percentages = (data.isnull().sum() / len(data)) * 100
print(round(null_percentages,2).sort_values(ascending=True).head(30))

TransactionID    0.0
V291             0.0
V290             0.0
V289             0.0
V288             0.0
V287             0.0
V286             0.0
V285             0.0
V284             0.0
V283             0.0
V282             0.0
V281             0.0
V292             0.0
V280             0.0
V137             0.0
V136             0.0
V135             0.0
V134             0.0
V133             0.0
V132             0.0
V131             0.0
V130             0.0
V129             0.0
V128             0.0
V127             0.0
V279             0.0
V126             0.0
V293             0.0
V295             0.0
V319             0.0
dtype: float64


In [12]:
data.shape

(57801, 112)

## Conversión de datos

No es necesatio convertir los tipos de datos, debido a que son adecuado al tipo de datos que ya tienen.

In [13]:
data.dtypes

TransactionID       int64
isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
                   ...   
V317              float64
V318              float64
V319              float64
V320              float64
V321              float64
Length: 112, dtype: object

## Separamos datos características (X) y variable objetivo (y)

In [14]:
X = data.drop('isFraud', axis=1)  # features
y = data['isFraud']  # target

## Creación de nuevas características (feature engineering).

In [15]:

# # Calcular los cuartiles y el IQR
# Q1 = np.percentile(X['TransactionAmt'], 25)
# Q3 = np.percentile(X['TransactionAmt'], 75)
# IQR = Q3 - Q1

# # Definir los umbrales para valores atípicos
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# # Función para clasificar los valores en rangos
# def classify_transaction_amt(value):
#     if value < lower_bound:
#         return 'Muy bajo'
#     elif lower_bound <= value < Q1:
#         return 'Bajo'
#     elif Q1 <= value < Q3:
#         return 'Medio'
#     elif Q3 <= value < upper_bound:
#         return 'Alto'
#     else:
#         return 'Muy alto'

# # Aplicar la función de clasificación a la columna
# X['TransactionAmt_Range'] = X['TransactionAmt'].apply(classify_transaction_amt)


# # Mostrar algunos resultados
# print(X['TransactionAmt_Range'].value_counts())


### Eliminamos la columna 'TransactionAmt' por la conclusiones obtenidas en EDA

In [16]:
#X.drop('TransactionAmt', axis=1, inplace=True)

### Juntar addr1 y addr2

- addr1 representa una región dentro de un país, mientras que addr2 corresponde al código de país. Combinar estas variables podría capturar de manera más efectiva la relación geográfica entre regiones y países en el modelo.

In [17]:
# X['addr1'] = X['addr1'].astype('int').astype('str')
# X['addr2'] = X['addr2'].astype('int').astype('str')

# # Concatenar addr1 y addr2 en una nueva columna addr_combined usando +
# X['addr_combined'] = X['addr1'] + '_' + X['addr2']

In [18]:
# X.columns

In [19]:
# ## Eliminamos addr1 y addr2 de X
# X.drop(['addr1', 'addr2'],axis=1, inplace=True)

### Codificación de variables categóricas

In [20]:
X.dtypes

TransactionID       int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
card1               int64
                   ...   
V317              float64
V318              float64
V319              float64
V320              float64
V321              float64
Length: 111, dtype: object

In [21]:
# Identificar tipos de datos
data_types = X.dtypes

# Filtrar y contar variables numéricas y categóricas
num_vars = data_types[data_types != 'object']  # Variables numéricas
cat_vars = data_types[data_types == 'object']  # Variables categóricas
num_vars_names = num_vars.index.tolist()
cat_vars_names = cat_vars.index.tolist()

In [22]:
for columna in cat_vars_names :
    print(f"Columna: {columna}, el número de variables es: {X[columna].nunique()}")

Columna: ProductCD, el número de variables es: 5
Columna: card4, el número de variables es: 4
Columna: card6, el número de variables es: 3


Debido a que cada columna tiene diferentes tipos de categorías, aplicaremos según sea adecuada a cada cado.
- ProductCD (4 categorías):
    - One-Hot Encoding: Como ProductCD tiene un número pequeño y fijo de categorías (4 en total), el one-hot encoding es una opción adecuada.

- TransactionAmt_Range (4 categorías):
    - Dado que TransactionAmt_Range tiene un orden natural o ordinal con 4 categorías, el ordinal encoding es apropiado. Asigna valores numéricos secuenciales a cada categoría.
- P_emaildomain (56 categorías) y  addr_combined:
    - Hashing Trick o Binary Encoding:  Dado que P_emaildomain tiene muchas categorías (56), el one-hot encoding puede generar demasiadas columnas y aumentar la complejidad. binary encoding podrían ser más eficientes.


In [24]:
X.select_dtypes(include=['object']).nunique()

ProductCD    5
card4        4
card6        3
dtype: int64

In [25]:
def one_hot_encode(df, columns):
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(df[columns])
    encoded_df = pd.DataFrame(encoded.toarray(),
                              columns=encoder.get_feature_names_out(columns),
                              index=df.index)
    df = df.drop(columns, axis=1)
    return pd.concat([df, encoded_df], axis=1)

def binary_encode(df, columns):
    encoder = ce.BinaryEncoder(cols=columns)
    encoded = encoder.fit_transform(df[columns])
    df = df.drop(columns, axis=1)
    return pd.concat([df, encoded], axis=1)

# Codificación One-Hot para ProductCD, card4 y card6
X = one_hot_encode(X, ['ProductCD', 'card4', 'card6'])

# # Codificación BinaryEncoder para P_emaildomain
# X = binary_encode(X, ['P_emaildomain'])

# # Codificación BinaryEncoder para addr_combined
# X = binary_encode(X, ['addr_combined'])


In [ ]:
X.columns

## Normalización y estandarización de las variables.

- Basados en el anális EDA la columna 'TransactionDT' tiene una distribución casi uniforme por lo tanto la normalización Min-Max ajusta los valores de una característica a un rango específico, típicamente entre 0 y 1. Es útil cuando deseas mantener la distribución de los datos pero ajustarla a una escala uniforme.

In [38]:
X[num_vars.index.tolist()]

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
294161,3281161,7257076,40.000,16659,170.0,150.0,226.0,2.0,1.0,0.0,...,0.000000,0.0,0.0,0.0,80.0,80.0,80.0,0.0,0.0,0.0
467048,3454048,12019989,46.417,11201,103.0,185.0,226.0,2.0,2.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34825,3021825,855320,75.000,8131,583.0,150.0,224.0,1.0,1.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
374859,3361859,9335149,38.950,7508,321.0,150.0,226.0,103.0,102.0,0.0,...,311.950012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
545712,3532712,14406961,26.500,16132,111.0,150.0,226.0,1.0,1.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460423,3447423,11829608,131.000,9500,321.0,150.0,226.0,3.0,3.0,0.0,...,131.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
364785,3351785,9059347,77.000,14382,595.0,150.0,195.0,1.0,1.0,0.0,...,160.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
157056,3144056,3270520,50.000,8378,499.0,150.0,226.0,2.0,2.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
523532,3510532,13743340,213.000,12559,555.0,150.0,226.0,2.0,1.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
V = [ 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115', 'V116', 'V117', 'V118', 'V119', 'V120', 'V121', 'V122', 'V123', 'V124', 'V125', 'V126', 'V127', 'V128', 'V129', 'V130', 'V131', 'V132', 'V133', 'V134', 'V135', 'V136', 'V137', 'V279', 'V280', 'V281', 'V282', 'V283', 'V284', 'V285', 'V286', 'V287', 'V288', 'V289', 'V290', 'V291', 'V292', 'V293', 'V294', 'V295', 'V296', 'V297', 'V298', 'V299', 'V300', 'V301', 'V302', 'V303', 'V304', 'V305', 'V306', 'V307', 'V308', 'V309', 'V310', 'V311', 'V312', 'V313', 'V314', 'V315', 'V316', 'V317', 'V318', 'V319', 'V320', 'V321']

In [41]:
# Aplicar MinMaxScaler a las columnas específicas
scaler_min_max = MinMaxScaler()
X[['TransactionDT', 'card1', 'card2', 'card3', 'card5']] = scaler_min_max.fit_transform(X[['TransactionDT', 'card1', 'card2', 'card3', 'card5']])


# Aplicar RobustScaler a 'TransactionAmt'
scaler_robust = RobustScaler()
X[['TransactionAmt','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14']] = scaler_robust.fit_transform(X[['TransactionAmt','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14']])

# Aplicar StandardScaler a 'dist1'
scaler_standard = StandardScaler()
X[V] = scaler_standard.fit_transform(X[V])


In [42]:
X.head()

,TransactionID,TransactionDT,TransactionAmt,card1,card2,card3,card5,C1,C2,C3,...,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,card6_credit,card6_debit,card6_debit or credit
294161,3281161,0.456027,-0.347349,0.900155,0.140,0.387597,0.919708,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
467048,3454048,0.758933,-0.269141,0.586243,0.006,0.658915,0.919708,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34825,3021825,0.048896,0.079220,0.409674,0.966,0.387597,0.905109,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
374859,3361859,0.588186,-0.360146,0.373843,0.442,0.387597,0.919708,51.0,50.5,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
545712,3532712,0.910736,-0.511883,0.869845,0.022,0.387597,0.919708,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [43]:
# Guardar X y y en archivos CSV

#dir_data_processed = '../data/processed'
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'datos_procesados.csv')
X.to_csv(ruta_archivo_X, index=False, encoding='utf-8')

ruta_archivo_y = os.path.join(dir_data_processed, 'target.csv')
y.to_csv(ruta_archivo_y, index=False, encoding='utf-8')



## Conclusión

En este proyecto de Data Science, el proceso de limpieza y preprocesamiento de datos ha sido fundamental para preparar nuestros datos antes de aplicar modelos de aprendizaje automático. A continuación, se detallan las principales etapas y decisiones tomadas durante este proceso:

1. **Selección Estratégica del Dataset**: Se cargó el dataset completo y se seleccionó estratégicamente una porción del 10%, utilizando muestreo estratificado para asegurar representatividad y evitar sesgos en nuestros modelos.

2. **Selección de Características Relevantes**: Se llevó a cabo la selección de las columnas más relevantes para nuestro análisis y modelos, descartando aquellas que no contribuían significativamente a la predicción del target.

3. **Manejo de Valores Nulos**: Se eliminaron aquellas columnas que contenían un alto porcentaje (mayor al 80%) de valores nulos, así como las filas con un porcentaje bajo de valores nulos para garantizar la integridad de los datos restantes.

4. **Separación de Variables**: Se realizó una clara separación entre las variables características (`X`) y el target (`y`), asegurando que estuvieran correctamente definidas para el entrenamiento de los modelos.

5. **Ingeniería de Características**: Basados en un análisis exploratorio de datos (EDA), se reemplazó la columna `TransactionAmt` por `TransactionAmt_Range`, una variable categórica que agrupa los valores en rangos como "muy bajo", "bajo", "medio", "alto" y "muy alto". Esta transformación facilita el manejo de variables con amplios rangos de valores, evitando posibles complicaciones durante el entrenamiento de los modelos.

6. **Combinación de Variables**: Se combinaron las columnas `addr1` y `addr2` en una sola columna (`addr_combined`), reduciendo así la dimensionalidad del dataset sin perder información relevante.

7. **Codificación de Variables Categóricas**: Se aplicaron técnicas adecuadas de codificación a las variables categóricas según su naturaleza, como codificación one-hot y binary encoding, para prepararlas para su uso en los modelos de aprendizaje automático.

8. **Normalización de Variables Numéricas**: Se normalizaron las variables numéricas para asegurar que todas estuvieran en la misma escala, lo cual es crucial para modelos que se basan en la distancia o magnitud de los atributos.

En resumen, el proceso de limpieza y preprocesamiento de datos realizado ha permitido transformar el dataset inicial en un formato apto y optimizado para la construcción de modelos predictivos. Estas etapas son fundamentales para asegurar la calidad de los resultados obtenidos y facilitar la interpretación y aplicación de los modelos en la práctica.
